# DC Child Tax Credit Reform Analysis

This notebook analyzes baseline and reform scenarios for the DC Child Tax Credit at various cost levels ($5M, $10M, and $15M) targeting different age groups:
- Preschool age (under 4)
- Early childhood (under 6)
- All children (under 18)

In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import numpy as np

c:\Users\dtsax\envs\pe\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load DC dataset (same as dataset_exploration notebook)
sim = Microsimulation(dataset='hf://policyengine/test/DC.h5')

## Baseline: Zero Out Current DC CTC

Current DC CTC provides $420 per child under age 6, with income phase-outs starting at $160k (single)/$240k (joint).

In [3]:
def create_baseline_reform():
    """Zero out the existing DC CTC"""
    return Reform.from_dict(
        {
            "gov.states.dc.tax.income.credits.ctc.amount": {
                "2025-01-01.2100-12-31": 0
            }
        },
        country_id="us",
    )

baseline_reform = create_baseline_reform()
baseline_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=baseline_reform)

In [4]:
# Calculate baseline cost
current_ctc = sim.calculate("dc_ctc", period=2025, map_to="household")
baseline_ctc = baseline_sim.calculate("dc_ctc", period=2025, map_to="household")

current_cost = current_ctc.sum()
print(f"Current DC CTC annual cost: ${current_cost:,.0f}")
print(f"Baseline DC CTC cost (zeroed out): ${baseline_ctc.sum():,.0f}")

Current DC CTC annual cost: $17,971,915
Baseline DC CTC cost (zeroed out): $0


In [5]:
def create_preschool_reform(amount_per_child):
    """Create reform targeting preschool age children (under 4)"""
    return Reform.from_dict(
        {
            "gov.states.dc.tax.income.credits.ctc.amount": {
                "2025-01-01.2100-12-31": amount_per_child
            },
            "gov.states.dc.tax.income.credits.ctc.child.age_threshold": {
                "2025-01-01.2100-12-31": 4
            }
        },
        country_id="us",
    )

# Adjusted estimates based on actual costs
preschool_5m_amount = 175  # Estimate for $5M
preschool_10m_amount = 350  # Estimate for $10M
preschool_15m_amount = 525  # Estimate for $15M

preschool_5m_reform = create_preschool_reform(preschool_5m_amount)
preschool_10m_reform = create_preschool_reform(preschool_10m_amount)
preschool_15m_reform = create_preschool_reform(preschool_15m_amount)

preschool_5m_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=preschool_5m_reform)
preschool_10m_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=preschool_10m_reform)
preschool_15m_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=preschool_15m_reform)

In [6]:
# Calculate costs for preschool reforms
preschool_5m_ctc = preschool_5m_sim.calculate("dc_ctc", period=2025, map_to="household")
preschool_10m_ctc = preschool_10m_sim.calculate("dc_ctc", period=2025, map_to="household")
preschool_15m_ctc = preschool_15m_sim.calculate("dc_ctc", period=2025, map_to="household")

print("Reform 1: Preschool Age (Under 4)")
print(f"  $5M target - Amount per child: ${preschool_5m_amount}, Actual cost: ${preschool_5m_ctc.sum():,.0f}")
print(f"  $10M target - Amount per child: ${preschool_10m_amount}, Actual cost: ${preschool_10m_ctc.sum():,.0f}")
print(f"  $15M target - Amount per child: ${preschool_15m_amount}, Actual cost: ${preschool_15m_ctc.sum():,.0f}")

Reform 1: Preschool Age (Under 4)
  $5M target - Amount per child: $175, Actual cost: $5,017,059
  $10M target - Amount per child: $350, Actual cost: $10,068,140
  $15M target - Amount per child: $525, Actual cost: $15,148,594


In [7]:
def create_under_6_reform(amount_per_child):
    """Create reform for children under 6 (current age threshold)"""
    return Reform.from_dict(
        {
            "gov.states.dc.tax.income.credits.ctc.amount": {
                "2025-01-01.2100-12-31": amount_per_child
            },
            "gov.states.dc.tax.income.credits.ctc.child.age_threshold": {
                "2025-01-01.2100-12-31": 6
            }
        },
        country_id="us",
    )

# Adjusted estimates based on actual costs
under6_5m_amount = 120  # Estimate for $5M
under6_10m_amount = 235  # Estimate for $10M
under6_15m_amount = 350  # Estimate for $15M

under6_5m_reform = create_under_6_reform(under6_5m_amount)
under6_10m_reform = create_under_6_reform(under6_10m_amount)
under6_15m_reform = create_under_6_reform(under6_15m_amount)

under6_5m_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=under6_5m_reform)
under6_10m_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=under6_10m_reform)
under6_15m_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=under6_15m_reform)

In [8]:
# Calculate costs for under 6 reforms
under6_5m_ctc = under6_5m_sim.calculate("dc_ctc", period=2025, map_to="household")
under6_10m_ctc = under6_10m_sim.calculate("dc_ctc", period=2025, map_to="household")
under6_15m_ctc = under6_15m_sim.calculate("dc_ctc", period=2025, map_to="household")

print("Reform 2: Early Childhood (Under 6)")
print(f"  $5M target - Amount per child: ${under6_5m_amount}, Actual cost: ${under6_5m_ctc.sum():,.0f}")
print(f"  $10M target - Amount per child: ${under6_10m_amount}, Actual cost: ${under6_10m_ctc.sum():,.0f}")
print(f"  $15M target - Amount per child: ${under6_15m_amount}, Actual cost: ${under6_15m_ctc.sum():,.0f}")

Reform 2: Early Childhood (Under 6)
  $5M target - Amount per child: $120, Actual cost: $5,104,786
  $10M target - Amount per child: $235, Actual cost: $10,019,970
  $15M target - Amount per child: $350, Actual cost: $14,957,336


In [9]:
def create_under_18_reform(amount_per_child):
    """Create reform for all children under 18"""
    return Reform.from_dict(
        {
            "gov.states.dc.tax.income.credits.ctc.amount": {
                "2025-01-01.2100-12-31": amount_per_child
            },
            "gov.states.dc.tax.income.credits.ctc.child.age_threshold": {
                "2025-01-01.2100-12-31": 18
            }
        },
        country_id="us",
    )

# Adjusted estimates based on actual costs
under18_5m_amount = 45  # Estimate for $5M
under18_10m_amount = 85  # Estimate for $10M
under18_15m_amount = 125  # Estimate for $15M

under18_5m_reform = create_under_18_reform(under18_5m_amount)
under18_10m_reform = create_under_18_reform(under18_10m_amount)
under18_15m_reform = create_under_18_reform(under18_15m_amount)

under18_5m_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=under18_5m_reform)
under18_10m_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=under18_10m_reform)
under18_15m_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=under18_15m_reform)

In [10]:
# Calculate costs for under 18 reforms
under18_5m_ctc = under18_5m_sim.calculate("dc_ctc", period=2025, map_to="household")
under18_10m_ctc = under18_10m_sim.calculate("dc_ctc", period=2025, map_to="household")
under18_15m_ctc = under18_15m_sim.calculate("dc_ctc", period=2025, map_to="household")

print("Reform 3: All Children (Under 18)")
print(f"  $5M target - Amount per child: ${under18_5m_amount}, Actual cost: ${under18_5m_ctc.sum():,.0f}")
print(f"  $10M target - Amount per child: ${under18_10m_amount}, Actual cost: ${under18_10m_ctc.sum():,.0f}")
print(f"  $15M target - Amount per child: ${under18_15m_amount}, Actual cost: ${under18_15m_ctc.sum():,.0f}")

Reform 3: All Children (Under 18)
  $5M target - Amount per child: $45, Actual cost: $5,302,008
  $10M target - Amount per child: $85, Actual cost: $10,026,294
  $15M target - Amount per child: $125, Actual cost: $14,760,191


In [11]:
def create_preschool_reform_lower_threshold(amount_per_child):
    """Create reform for children under 4 with lower income thresholds"""
    return Reform.from_dict(
        {
            "gov.states.dc.tax.income.credits.ctc.amount": {
                "2025-01-01.2100-12-31": amount_per_child
            },
            "gov.states.dc.tax.income.credits.ctc.child.age_threshold": {
                "2025-01-01.2100-12-31": 4
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SINGLE": {
                "2025-01-01.2100-12-31": 80_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.JOINT": {
                "2025-01-01.2100-12-31": 120_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SEPARATE": {
                "2025-01-01.2100-12-31": 60_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.HEAD_OF_HOUSEHOLD": {
                "2025-01-01.2100-12-31": 80_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SURVIVING_SPOUSE": {
                "2025-01-01.2100-12-31": 80_000
            }
        },
        country_id="us",
    )

# Adjusted amounts based on target costs
preschool_lower_5m_amount = 195   # Estimate for $5M
preschool_lower_10m_amount = 390  # Estimate for $10M
preschool_lower_15m_amount = 570  # Estimate for $15M

preschool_lower_5m_reform = create_preschool_reform_lower_threshold(preschool_lower_5m_amount)
preschool_lower_10m_reform = create_preschool_reform_lower_threshold(preschool_lower_10m_amount)
preschool_lower_15m_reform = create_preschool_reform_lower_threshold(preschool_lower_15m_amount)

preschool_lower_5m_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=preschool_lower_5m_reform)
preschool_lower_10m_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=preschool_lower_10m_reform)
preschool_lower_15m_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=preschool_lower_15m_reform)

In [12]:
# Calculate costs for preschool reforms with lower thresholds
preschool_lower_5m_ctc = preschool_lower_5m_sim.calculate("dc_ctc", period=2025, map_to="household")
preschool_lower_10m_ctc = preschool_lower_10m_sim.calculate("dc_ctc", period=2025, map_to="household")
preschool_lower_15m_ctc = preschool_lower_15m_sim.calculate("dc_ctc", period=2025, map_to="household")

print("Reform 1B: Preschool Age (Under 4) - Lower Income Thresholds")
print(f"  $5M target - Amount per child: ${preschool_lower_5m_amount}, Actual cost: ${preschool_lower_5m_ctc.sum():,.0f}")
print(f"  $10M target - Amount per child: ${preschool_lower_10m_amount}, Actual cost: ${preschool_lower_10m_ctc.sum():,.0f}")
print(f"  $15M target - Amount per child: ${preschool_lower_15m_amount}, Actual cost: ${preschool_lower_15m_ctc.sum():,.0f}")

Reform 1B: Preschool Age (Under 4) - Lower Income Thresholds
  $5M target - Amount per child: $195, Actual cost: $4,950,844
  $10M target - Amount per child: $390, Actual cost: $9,977,008
  $15M target - Amount per child: $570, Actual cost: $14,794,358


In [13]:
def create_under_6_reform_lower_threshold(amount_per_child):
    """Create reform for children under 6 with lower income thresholds"""
    return Reform.from_dict(
        {
            "gov.states.dc.tax.income.credits.ctc.amount": {
                "2025-01-01.2100-12-31": amount_per_child
            },
            "gov.states.dc.tax.income.credits.ctc.child.age_threshold": {
                "2025-01-01.2100-12-31": 6
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SINGLE": {
                "2025-01-01.2100-12-31": 80_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.JOINT": {
                "2025-01-01.2100-12-31": 120_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SEPARATE": {
                "2025-01-01.2100-12-31": 60_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.HEAD_OF_HOUSEHOLD": {
                "2025-01-01.2100-12-31": 80_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SURVIVING_SPOUSE": {
                "2025-01-01.2100-12-31": 80_000
            }
        },
        country_id="us",
    )

# Adjusted amounts based on target costs
under6_lower_5m_amount = 130   # Estimate for $5M
under6_lower_10m_amount = 265  # Estimate for $10M
under6_lower_15m_amount = 385  # Estimate for $15M

under6_lower_5m_reform = create_under_6_reform_lower_threshold(under6_lower_5m_amount)
under6_lower_10m_reform = create_under_6_reform_lower_threshold(under6_lower_10m_amount)
under6_lower_15m_reform = create_under_6_reform_lower_threshold(under6_lower_15m_amount)

under6_lower_5m_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=under6_lower_5m_reform)
under6_lower_10m_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=under6_lower_10m_reform)
under6_lower_15m_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=under6_lower_15m_reform)

In [14]:
# Calculate costs for under 6 reforms with lower thresholds
under6_lower_5m_ctc = under6_lower_5m_sim.calculate("dc_ctc", period=2025, map_to="household")
under6_lower_10m_ctc = under6_lower_10m_sim.calculate("dc_ctc", period=2025, map_to="household")
under6_lower_15m_ctc = under6_lower_15m_sim.calculate("dc_ctc", period=2025, map_to="household")

print("Reform 2B: Early Childhood (Under 6) - Lower Income Thresholds")
print(f"  $5M target - Amount per child: ${under6_lower_5m_amount}, Actual cost: ${under6_lower_5m_ctc.sum():,.0f}")
print(f"  $10M target - Amount per child: ${under6_lower_10m_amount}, Actual cost: ${under6_lower_10m_ctc.sum():,.0f}")
print(f"  $15M target - Amount per child: ${under6_lower_15m_amount}, Actual cost: ${under6_lower_15m_ctc.sum():,.0f}")

Reform 2B: Early Childhood (Under 6) - Lower Income Thresholds
  $5M target - Amount per child: $130, Actual cost: $4,910,300
  $10M target - Amount per child: $265, Actual cost: $10,060,217
  $15M target - Amount per child: $385, Actual cost: $14,728,679


In [15]:
def create_under_18_reform_lower_threshold(amount_per_child):
    """Create reform for all children under 18 with lower income thresholds"""
    return Reform.from_dict(
        {
            "gov.states.dc.tax.income.credits.ctc.amount": {
                "2025-01-01.2100-12-31": amount_per_child
            },
            "gov.states.dc.tax.income.credits.ctc.child.age_threshold": {
                "2025-01-01.2100-12-31": 18
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SINGLE": {
                "2025-01-01.2100-12-31": 80_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.JOINT": {
                "2025-01-01.2100-12-31": 120_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SEPARATE": {
                "2025-01-01.2100-12-31": 60_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.HEAD_OF_HOUSEHOLD": {
                "2025-01-01.2100-12-31": 80_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SURVIVING_SPOUSE": {
                "2025-01-01.2100-12-31": 80_000
            }
        },
        country_id="us",
    )

# Adjusted amounts based on target costs
under18_lower_5m_amount = 50   # Estimate for $5M
under18_lower_10m_amount = 95  # Estimate for $10M
under18_lower_15m_amount = 145  # Estimate for $15M

under18_lower_5m_reform = create_under_18_reform_lower_threshold(under18_lower_5m_amount)
under18_lower_10m_reform = create_under_18_reform_lower_threshold(under18_lower_10m_amount)
under18_lower_15m_reform = create_under_18_reform_lower_threshold(under18_lower_15m_amount)

under18_lower_5m_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=under18_lower_5m_reform)
under18_lower_10m_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=under18_lower_10m_reform)
under18_lower_15m_sim = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=under18_lower_15m_reform)

In [16]:
# Calculate costs for under 18 reforms with lower thresholds
under18_lower_5m_ctc = under18_lower_5m_sim.calculate("dc_ctc", period=2025, map_to="household")
under18_lower_10m_ctc = under18_lower_10m_sim.calculate("dc_ctc", period=2025, map_to="household")
under18_lower_15m_ctc = under18_lower_15m_sim.calculate("dc_ctc", period=2025, map_to="household")

print("Reform 3B: All Children (Under 18) - Lower Income Thresholds")
print(f"  $5M target - Amount per child: ${under18_lower_5m_amount}, Actual cost: ${under18_lower_5m_ctc.sum():,.0f}")
print(f"  $10M target - Amount per child: ${under18_lower_10m_amount}, Actual cost: ${under18_lower_10m_ctc.sum():,.0f}")
print(f"  $15M target - Amount per child: ${under18_lower_15m_amount}, Actual cost: ${under18_lower_15m_ctc.sum():,.0f}")

Reform 3B: All Children (Under 18) - Lower Income Thresholds
  $5M target - Amount per child: $50, Actual cost: $5,147,592
  $10M target - Amount per child: $95, Actual cost: $9,823,450
  $15M target - Amount per child: $145, Actual cost: $15,059,910


In [17]:
# Create summary DataFrame including all reforms
summary_data = [
    {
        "Reform": "Baseline (Current)",
        "Age Group": "Under 6",
        "Income Threshold": "Current",
        "Target Cost": "Current",
        "Amount per Child": "$420",
        "Actual Cost": f"${current_cost:,.0f}"
    },
    # Reform 1: Preschool
    {
        "Reform": "Reform 1: Preschool",
        "Age Group": "Under 4",
        "Income Threshold": "Standard",
        "Target Cost": "$5M",
        "Amount per Child": f"${preschool_5m_amount}",
        "Actual Cost": f"${preschool_5m_ctc.sum():,.0f}"
    },
    {
        "Reform": "Reform 1: Preschool",
        "Age Group": "Under 4",
        "Income Threshold": "Standard",
        "Target Cost": "$10M",
        "Amount per Child": f"${preschool_10m_amount}",
        "Actual Cost": f"${preschool_10m_ctc.sum():,.0f}"
    },
    {
        "Reform": "Reform 1: Preschool",
        "Age Group": "Under 4",
        "Income Threshold": "Standard",
        "Target Cost": "$15M",
        "Amount per Child": f"${preschool_15m_amount}",
        "Actual Cost": f"${preschool_15m_ctc.sum():,.0f}"
    },
    # Reform 1B: Preschool Lower Threshold
    {
        "Reform": "Reform 1B: Preschool",
        "Age Group": "Under 4",
        "Income Threshold": "Half",
        "Target Cost": "$5M",
        "Amount per Child": f"${preschool_lower_5m_amount}",
        "Actual Cost": f"${preschool_lower_5m_ctc.sum():,.0f}"
    },
    {
        "Reform": "Reform 1B: Preschool",
        "Age Group": "Under 4",
        "Income Threshold": "Half",
        "Target Cost": "$10M",
        "Amount per Child": f"${preschool_lower_10m_amount}",
        "Actual Cost": f"${preschool_lower_10m_ctc.sum():,.0f}"
    },
    {
        "Reform": "Reform 1B: Preschool",
        "Age Group": "Under 4",
        "Income Threshold": "Half",
        "Target Cost": "$15M",
        "Amount per Child": f"${preschool_lower_15m_amount}",
        "Actual Cost": f"${preschool_lower_15m_ctc.sum():,.0f}"
    },
    # Reform 2: Early Childhood
    {
        "Reform": "Reform 2: Early Childhood",
        "Age Group": "Under 6",
        "Income Threshold": "Standard",
        "Target Cost": "$5M",
        "Amount per Child": f"${under6_5m_amount}",
        "Actual Cost": f"${under6_5m_ctc.sum():,.0f}"
    },
    {
        "Reform": "Reform 2: Early Childhood",
        "Age Group": "Under 6",
        "Income Threshold": "Standard",
        "Target Cost": "$10M",
        "Amount per Child": f"${under6_10m_amount}",
        "Actual Cost": f"${under6_10m_ctc.sum():,.0f}"
    },
    {
        "Reform": "Reform 2: Early Childhood",
        "Age Group": "Under 6",
        "Income Threshold": "Standard",
        "Target Cost": "$15M",
        "Amount per Child": f"${under6_15m_amount}",
        "Actual Cost": f"${under6_15m_ctc.sum():,.0f}"
    },
    # Reform 2B: Early Childhood Lower Threshold
    {
        "Reform": "Reform 2B: Early Childhood",
        "Age Group": "Under 6",
        "Income Threshold": "Half",
        "Target Cost": "$5M",
        "Amount per Child": f"${under6_lower_5m_amount}",
        "Actual Cost": f"${under6_lower_5m_ctc.sum():,.0f}"
    },
    {
        "Reform": "Reform 2B: Early Childhood",
        "Age Group": "Under 6",
        "Income Threshold": "Half",
        "Target Cost": "$10M",
        "Amount per Child": f"${under6_lower_10m_amount}",
        "Actual Cost": f"${under6_lower_10m_ctc.sum():,.0f}"
    },
    {
        "Reform": "Reform 2B: Early Childhood",
        "Age Group": "Under 6",
        "Income Threshold": "Half",
        "Target Cost": "$15M",
        "Amount per Child": f"${under6_lower_15m_amount}",
        "Actual Cost": f"${under6_lower_15m_ctc.sum():,.0f}"
    },
    # Reform 3: All Children
    {
        "Reform": "Reform 3: All Children",
        "Age Group": "Under 18",
        "Income Threshold": "Standard",
        "Target Cost": "$5M",
        "Amount per Child": f"${under18_5m_amount}",
        "Actual Cost": f"${under18_5m_ctc.sum():,.0f}"
    },
    {
        "Reform": "Reform 3: All Children",
        "Age Group": "Under 18",
        "Income Threshold": "Standard",
        "Target Cost": "$10M",
        "Amount per Child": f"${under18_10m_amount}",
        "Actual Cost": f"${under18_10m_ctc.sum():,.0f}"
    },
    {
        "Reform": "Reform 3: All Children",
        "Age Group": "Under 18",
        "Income Threshold": "Standard",
        "Target Cost": "$15M",
        "Amount per Child": f"${under18_15m_amount}",
        "Actual Cost": f"${under18_15m_ctc.sum():,.0f}"
    },
    # Reform 3B: All Children Lower Threshold
    {
        "Reform": "Reform 3B: All Children",
        "Age Group": "Under 18",
        "Income Threshold": "Half",
        "Target Cost": "$5M",
        "Amount per Child": f"${under18_lower_5m_amount}",
        "Actual Cost": f"${under18_lower_5m_ctc.sum():,.0f}"
    },
    {
        "Reform": "Reform 3B: All Children",
        "Age Group": "Under 18",
        "Income Threshold": "Half",
        "Target Cost": "$10M",
        "Amount per Child": f"${under18_lower_10m_amount}",
        "Actual Cost": f"${under18_lower_10m_ctc.sum():,.0f}"
    },
    {
        "Reform": "Reform 3B: All Children",
        "Age Group": "Under 18",
        "Income Threshold": "Half",
        "Target Cost": "$15M",
        "Amount per Child": f"${under18_lower_15m_amount}",
        "Actual Cost": f"${under18_lower_15m_ctc.sum():,.0f}"
    }
]

summary_df = pd.DataFrame(summary_data)
print("\nDC Child Tax Credit Reform Summary")
print("="*100)
print(summary_df.to_string(index=False))


DC Child Tax Credit Reform Summary
                    Reform Age Group Income Threshold Target Cost Amount per Child Actual Cost
        Baseline (Current)   Under 6          Current     Current             $420 $17,971,915
       Reform 1: Preschool   Under 4         Standard         $5M             $175  $5,017,059
       Reform 1: Preschool   Under 4         Standard        $10M             $350 $10,068,140
       Reform 1: Preschool   Under 4         Standard        $15M             $525 $15,148,594
      Reform 1B: Preschool   Under 4             Half         $5M             $195  $4,950,844
      Reform 1B: Preschool   Under 4             Half        $10M             $390  $9,977,008
      Reform 1B: Preschool   Under 4             Half        $15M             $570 $14,794,358
 Reform 2: Early Childhood   Under 6         Standard         $5M             $120  $5,104,786
 Reform 2: Early Childhood   Under 6         Standard        $10M             $235 $10,019,970
 Reform 2: Ear

In [18]:
# Export to CSV
summary_df.to_csv("dc_ctc_reform_summary.csv", index=False)
print("\nExported to dc_ctc_reform_summary.csv")


Exported to dc_ctc_reform_summary.csv
